In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import pickle

from datetime import datetime, date, time
from textblob import TextBlob
from textblob import Word
from pre_processing import *
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Andres
[nltk_data]     Montero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stop = stopwords.words('english')
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]


In [3]:
DATA_FOLDER = './data'

tweets_col_names=['text']

tweets_dtypes = {'text': str }

cleaned_pos = True
cleaned_neg = True
cleaned_test = True

In [4]:
#verify  flag for data already cleaned
if not cleaned_neg :
    #read data from .txt to preprocess
    tweets_neg = pd.read_fwf(DATA_FOLDER + '/train_neg.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    
    #increase index to start at 1
    tweets_neg.index = tweets_neg.index +1
    #remove duplicates
    tweets_neg.drop_duplicates(inplace=True)
    #apply pre_process funtioncs
    #remove stop words
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    #separate number with letters  1234test123 => 1234 test 123
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: split_number_text(x))
    #filter digits
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: filter_digits(x))
    #remove hashtag #
    tweets_neg['text'] = tweets_neg['text'].str.replace('#', '')
    #replace ur by your
    tweets_neg['text'] = tweets_neg['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: expand_contractions(x))
    # interpret emoji
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: interpret_emoji(x))
    # remove punctuaction ......=> 
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_punctuation(x))
    # remove words user, url, number
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaapy => hapy
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_neg = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[:10]
    freq_max_neg = list(freq_max_neg.index)
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_neg))
    
    #remove least frequent words
    freq_min_neg = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[-10:]
    freq_min_neg = list(freq_min_neg.index)
    tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_neg))

    #correct words with textblob
    # tweets_neg['text'] = tweets_neg['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_neg
    
    #lemmatize words
    tweets_neg['text_lema'] = tweets_neg['text'].apply(lemmatize_text)
    
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_neg.pickle","wb")
    pickle.dump(tweets_neg['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
else:
    print("Already Trained!")
    pickle_in = open("data/pre_processed/tweets_neg.pickle","rb")
    tweets_neg_pickle = pickle.load(pickle_in)
    print("Opening pickle")
    tweets_neg_pickle
    

Already Trained!
Opening pickle


In [5]:
if not cleaned_pos:
    #read data .txt file
    tweets_pos = pd.read_fwf(DATA_FOLDER + '/train_pos.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
    # index start at 1
    tweets_pos.index = tweets_pos.index + 1
    #remove duplicates
    tweets_pos.drop_duplicates(inplace=True)
    #apply pre_process funtioncs
    #remove stop words
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    #separate number with letters  1234test123 => 1234 test 123
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: split_number_text(x))
    #filter digits
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: filter_digits(x))
    #remove hashtag #
    tweets_pos['text'] = tweets_pos['text'].str.replace('#', '')
    #replace ur by your
    tweets_pos['text'] = tweets_pos['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: expand_contractions(x))
    # interpret emoji
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: interpret_emoji(x))
    # remove punctuaction ......=> 
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_punctuation(x))
    # remove words user, url, number
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaapy => hapy
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[:10]
    freq_max_pos = list(freq_max_pos.index)
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_pos))
    
    #remove least frequent words
    freq_min_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[-10:]
    freq_min_pos = list(freq_min_pos.index)
    tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_pos))

    #correct words with textblob
    # tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_pos
    
    #lemmatize words
    tweets_pos['text_lema'] = tweets_pos['text'].apply(lemmatize_text)
    
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_pos.pickle","wb")
    pickle.dump(tweets_pos['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
else:
    print("Already Trained!")
    pickle_in = open("data/pre_processed/tweets_pos.pickle","rb")
    tweets_pos_pickle = pickle.load(pickle_in)
    print("Opening pickle")
    tweets_pos_pickle

Already Trained!
Opening pickle


In [6]:
if not cleaned_test:  
    #read data from .txt
    test_data = pd.read_fwf(DATA_FOLDER + '/test_data.txt', header=None)
    #index start at 1
    test_data.index = test_data.index +1
    #remove duplicates
    test_data.drop_duplicates(inplace=True)
    test_data.drop(columns=[1,2], inplace=True)
    test_data.rename(columns={0:'text'}, inplace= True)
    
    #apply pre_process funtioncs
    #remove stop words
    test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    #separate number with letters  1234test123 => 1234 test 123
    test_data['text'] = test_data['text'].apply(lambda x: split_number_text(x))
    #filter digits
    test_data['text'] = test_data['text'].apply(lambda x: filter_digits(x))
    #remove hashtag #
    test_data['text'] = test_data['text'].str.replace('#', '')
    #replace ur by your
    test_data['text'] = test_data['text'].str.replace('ur', 'your')
    #expand contractions don't => do not
    test_data['text'] = test_data['text'].apply(lambda x: expand_contractions(x))
    # interpret emoji
    test_data['text'] = test_data['text'].apply(lambda x: interpret_emoji(x))
    # remove punctuaction ......=> 
    test_data['text'] = test_data['text'].apply(lambda x: remove_punctuation(x))
    # remove words user, url, number
    test_data['text'] = test_data['text'].apply(lambda x: remove_words(x))
    # replace more letters haaaaaaaaaapy => hapy
    test_data['text'] = test_data['text'].apply(lambda x: replace_moreletters(x))
    
    #remove most frequent words 
    freq_max_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[:10]
    freq_max_test = list(freq_max_test.index)
    test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_test))
    
    #remove least frequent words
    freq_min_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[-10:]
    freq_min_test = list(freq_min_test.index)
    test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_test))

    #correct words with textblob
    # tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    # tweets_pos
    
    #lemmatize words
    test_data['text_lema'] = test_data['text'].apply(lemmatize_text)

    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/test_data.pickle","wb")
    pickle.dump(test_data['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
else:
    print("Already Trained!")
    pickle_in = open("data/pre_processed/test_data.pickle","rb")
    test_data_pickle = pickle.load(pickle_in)
    print("Opening pickle")
    test_data_pickle

Already Trained!
Opening pickle


In [7]:
tweets_neg_pickle.head(20)

0     [into, tresorpack, difficulty, object, disasse...
1        [glad, dot, take, tomorrow, thankful, started]
2     [v, celtic, regular, season, tucked, play, pla...
3                  [could, actually, kill, girl, sorry]
4                     [find, hard, believe, in, afraid]
5                         [wish, could, night, tonight]
6                                     [got, kicked, wm]
7                        [yes, tell, lip, closed, okay]
8                                             [perfect]
9     [hi, harry, have, good, time, a, didn, see, ma...
10    [introduction, programming, +, +, nd, edition,...
13                                           [white, a]
14                   [dan, love, sad, wheresthegeneral]
15               [many, wonderful, building, do, still]
16                     [annoying, secretly, find, good]
17    [post, boom, spanish, american, fiction, sun, ...
18    [layer, heart, paperback, journey, inspired, r...
20                       [guess, tested, want, u

In [8]:
tweets_pos_pickle.head(20)

0     [funny, austin, read, mention, austin, god, kn...
1           [logic, dumb, even, crop, name, photo, ask]
2         [put, caper, box, looked, battle, crakkbitch]
3     [thanks, sir, trip, ll, mamma, keep, down, a, ...
4     [visiting, brother, mr, beset, birthday, gift,...
5     [may, lifecompleted, sweet, facebook, let, ple...
6     [dnextalbumtitle, feel, rollercoaster, life, s...
7     [working, hard, hardly, working, harder's, fut...
8                                  [saw, replying, bit]
9                                              [belong]
10                               [and, cheer, national]
11    [send, invitation, shop, line, find, everythin...
12                             [woke, finn, go, church]
13                             [agreed, day, left, the]
14                                    [money, katemelo]
15                [dam, alexis, got, lot, say, twitter]
16    [grateful, today, dream, fulfilled, heart, ful...
17                         [home, affair, shall,

In [9]:
test_data_pickle.head(20)

1     [sea, do, pro, sea, sooner, sport, portable, s...
2     [suck, well, work, week, come, cheer, oh, put,...
3                    [can, stay, away, bug, that, baby]
4     [madam, in, perfectly, fine, contagious, anymo...
5     [whenever, fall, asleep, watching, to, always,...
6     [need, get, rid, thing, scar, need, car, eithe...
7                            [whatever, terrible, mood]
8             [yes, thanks, jordan, donna, call, later]
9                    [friend, text, check, last, night]
10    [followback, please, unitytoyour, come, europe...
11         [watch, dumb, mass, get, lock, today, happy]
12    [obsessed, phase, killed, best, album, ever, n...
13    [robert, de, no, gay, name, vewy, would, under...
14              [canada, grade, since, grade, th, suck]
15     [please, say, hi, denmark, would, amazing, live]
16                                      [finally, home]
17    [custom, picture, poster, wide, complete, gold...
18            [new, follower, mention, followbac

In [ ]:
b = TextBlob("bout to be!")